### Spark-ML

1. Chain spark dataframe methods together to do data munging.
2. Be able to describe the Spark-ML API, and recognize differences to sk-learn.
3. Chain Spark-ML Transformers and Estimators together to compose ML pipelines.

In [1]:
import pyspark.sql.functions as F
import pyspark as ps
from pyspark import SQLContext  

spark = ps.sql.SparkSession.builder \
    .master('local[2]') \
    .appName('spark-ml') \
    .getOrCreate()

sc = spark.sparkContext
print("imported pyspark")

imported pyspark


In [2]:
sqlContext = SQLContext(sc)
print("defined sqlcontext")

defined sqlcontext


### Find the date on which AAPL's closing stock price was the highest

#### Input DataFrame



In [4]:

# read CSV
df_aapl = sqlContext.read.csv('data/aapl.csv',
                         header=True,       # use headers or not
                         quote='"',         # char for quotes
                         sep=",",           # char for separation
                         inferSchema=True)  # do we infer schema or not ?

df_aapl.show(5) #df.head(2)

+----------+----------+----------+----------+----------+----------+--------+
|      Date|      Open|      High|       Low|     Close| Adj Close|  Volume|
+----------+----------+----------+----------+----------+----------+--------+
|09/05/2018|186.550003|187.399994|185.220001|187.360001|186.640305|23211200|
|10/05/2018|187.740005|190.369995|187.649994|190.039993|189.309998|27989300|
|11/05/2018|189.490005|190.059998|187.449997|188.589996|188.589996|26212200|
|14/05/2018|189.009995|189.529999|187.860001|188.149994|188.149994|20778800|
|15/05/2018|186.779999|187.070007|185.100006|186.440002|186.440002|23695200|
+----------+----------+----------+----------+----------+----------+--------+
only showing top 5 rows

